In [1]:
import os
import tensorflow.lite as tflite
import numpy as np
from PIL import Image

In [2]:
def predict_plant_disease(image_path, model_path, label_path):
    # Load the TensorFlow Lite model
    interpreter = tflite.Interpreter(model_path)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Preprocess the image
    img = Image.open(image_path).resize((224, 224))
    img = np.array(img)
    img = img.astype(np.float32) / 255.0

    # Set the input tensor
    input_data = np.expand_dims(img, axis=0)
    input_data = input_data.astype(input_details[0]['dtype'])
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get the folder names as class labels
    label_names = sorted(os.listdir(label_path))

    # Get the output tensor
    output_tensor = interpreter.get_tensor(output_details[0]['index'])

    # Postprocess the output
    predicted_label_indices = np.argmax(output_tensor, axis=1)
    predicted_labels = [label_names[index] for index in predicted_label_indices]
    probabilities = output_tensor[0]

    return predicted_labels[0], probabilities, label_names

In [22]:
model_path = './assets/cauli_disease.tflite'
test_path = "./data\Plant Disease\Plants Disease Dataset Subset\Cauliflower/valid"
label_path = './data/Plant Disease/Plants Disease Dataset Subset/Cauliflower/train'
plants_type = "Cauliflower__Black_Rot"


folder_path = os.path.join(test_path, plants_type)
files = os.listdir(folder_path)
file_index = 4
file_path = os.path.join(folder_path, files[file_index])

predicted_label, prob, label = predict_plant_disease(file_path, model_path, label_path)
print("File \t:", plants_type)
print("Predicted label \t:", predicted_label)
print("Probabilities:")
for class_name, prob in zip(label, prob):
    print(f"{class_name} \t: {prob}")

File 	: Cauliflower__Black_Rot
Predicted label 	: Cauliflower__club_root
Probabilities:
Cauliflower__Alternaria_Leaf_Spot 	: 0.003351315390318632
Cauliflower__Bacterial_spot_rot 	: 0.18649044632911682
Cauliflower__Black_Rot 	: 0.1585872769355774
Cauliflower__Black_leg 	: 0.13751140236854553
Cauliflower__Downy_Mildew 	: 0.03515874966979027
Cauliflower__Powdery_mildew 	: 0.008177509531378746
Cauliflower__White_rust 	: 0.026118120178580284
Cauliflower__club_root 	: 0.2726014256477356
Cauliflower__healthy 	: 0.09315390139818192
Cauliflower__mosaic 	: 0.06162651628255844
Cauliflower__ring_spot 	: 0.01722337119281292
